In [1]:
import pandas as pd
import re
from geopy.geocoders import Nominatim

In [2]:
sales_houses = pd.read_csv("../inputs/houses_Madrid.csv")

In [3]:
sales_houses.head()

,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,...,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,...,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,4,21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,...,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,...,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,...,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,4,21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,...,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


In [4]:
# Vamos a comprobar las columnas por las que está compuesto nuestro DataFrame.

sales_houses.columns

Index(['Unnamed: 0', 'id', 'title', 'subtitle', 'sq_mt_built', 'sq_mt_useful',
       'n_rooms', 'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'latitude',
       'longitude', 'raw_address', 'is_exact_address_hidden', 'street_name',
       'street_number', 'portal', 'floor', 'is_floor_under', 'door',
       'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area',
       'is_rent_price_known', 'buy_price', 'buy_price_by_area',
       'is_buy_price_known', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'built_year', 'has_central_heating',
       'has_individual_heating', 'are_pets_allowed', 'has_ac',
       'has_fitted_wardrobes', 'has_lift', 'is_exterior', 'has_garden',
       'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room',
       'is_furnished', 'is_kitchen_equipped', 'is_accessible',
       'has_green_zones', 'energy_certificate', 'has_parking',
       'has_private_parking', 'has_public_parking',
       'is_parking_included_in_price', 'parki

Index(['Unnamed: 0', 'id', 'title', 'subtitle', 'sq_mt_built', 'sq_mt_useful',
       'n_rooms', 'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'latitude',
       'longitude', 'raw_address', 'is_exact_address_hidden', 'street_name',
       'street_number', 'portal', 'floor', 'is_floor_under', 'door',
       'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area',
       'is_rent_price_known', 'buy_price', 'buy_price_by_area',
       'is_buy_price_known', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'built_year', 'has_central_heating',
       'has_individual_heating', 'are_pets_allowed', 'has_ac',
       'has_fitted_wardrobes', 'has_lift', 'is_exterior', 'has_garden',
       'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room',
       'is_furnished', 'is_kitchen_equipped', 'is_accessible',
       'has_green_zones', 'energy_certificate', 'has_parking',
       'has_private_parking', 'has_public_parking',
       'is_parking_included_in_price', 'parki

In [5]:
# Seleccionamos aquellas columnas que más nos interesan para nuestro predicción de precios.

sales_houses = sales_houses[["neighborhood_id", "house_type_id", "is_renewal_needed", "is_new_development",
                             "sq_mt_built","n_rooms", "n_bathrooms", "floor", "is_exterior", "has_lift",
                             "has_parking", "has_storage_room", "has_terrace", "has_balcony", "has_garden", 
                             "has_pool", "buy_price", "buy_price_by_area"]]
sales_houses.head()

,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_garden,has_pool,buy_price,buy_price_by_area
0,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,HouseType 1: Pisos,False,False,64.0,2,1.0,3,True,False,False,NaN,NaN,NaN,NaN,NaN,85000,1328
1,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,HouseType 1: Pisos,True,False,70.0,3,1.0,4,True,True,False,NaN,True,NaN,NaN,NaN,129900,1856
2,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,94.0,2,2.0,1,True,True,False,True,NaN,NaN,NaN,NaN,144247,1535
3,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,True,True,False,True,NaN,NaN,NaN,NaN,109900,1717
4,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,HouseType 1: Pisos,False,False,108.0,2,2.0,4,True,True,True,True,NaN,NaN,NaN,True,260000,2407


,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_garden,has_pool,buy_price,buy_price_by_area
0,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,HouseType 1: Pisos,False,False,64.0,2,1.0,3,True,False,False,NaN,NaN,NaN,NaN,NaN,85000,1328
1,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,HouseType 1: Pisos,True,False,70.0,3,1.0,4,True,True,False,NaN,True,NaN,NaN,NaN,129900,1856
2,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,94.0,2,2.0,1,True,True,False,True,NaN,NaN,NaN,NaN,144247,1535
3,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,True,True,False,True,NaN,NaN,NaN,NaN,109900,1717
4,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,HouseType 1: Pisos,False,False,108.0,2,2.0,4,True,True,True,True,NaN,NaN,NaN,True,260000,2407


In [6]:
# Nuestro tasador se va a basar en viviendas plurifamiliares por lo que eliminaremos todas aquellas tipologías
# que no se correspondan con este tipo de viviendas.
# Vamos a comprobar por tanto que tipologías de viviendas tenemos en nuestro DataFrame.

sales_houses.house_type_id.value_counts()

HouseType 1: Pisos            17705
HouseType 2: Casa o chalet     1938
HouseType 5: Áticos            1032
HouseType 4: Dúplex             676
Name: house_type_id, dtype: int64

HouseType 1: Pisos            17705
HouseType 2: Casa o chalet     1938
HouseType 5: Áticos            1032
HouseType 4: Dúplex             676
Name: house_type_id, dtype: int64

In [7]:
# Eliminamos las viviendas de tipología tipo 2:

sales_houses = sales_houses.drop(sales_houses[sales_houses["house_type_id"]=="HouseType 5: Áticos"].index).reset_index()

In [8]:
# Vamos a comprobar ahora que tipo de datos tenemos 

sales_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20710 entries, 0 to 20709
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               20710 non-null  int64  
 1   neighborhood_id     20710 non-null  object 
 2   house_type_id       20319 non-null  object 
 3   is_renewal_needed   20710 non-null  bool   
 4   is_new_development  19740 non-null  object 
 5   sq_mt_built         20585 non-null  float64
 6   n_rooms             20710 non-null  int64  
 7   n_bathrooms         20694 non-null  float64
 8   floor               18196 non-null  object 
 9   is_exterior         17687 non-null  object 
 10  has_lift            18330 non-null  object 
 11  has_parking         20710 non-null  bool   
 12  has_storage_room    7155 non-null   object 
 13  has_terrace         8644 non-null   object 
 14  has_balcony         3185 non-null   object 
 15  has_garden          1556 non-null   object 
 16  has_

In [9]:
# Vamos a empezar por rellenar los valores NaN de aquellas columnas que tienen menos datos.
# Para ello, vamos a empezar comprobando los valores de dichas columnas

columnas = ["has_storage_room", "has_terrace", "has_balcony", "has_garden", "has_pool"]
for col in columnas:
    print(sales_houses[col].value_counts())

True    7155
Name: has_storage_room, dtype: int64
True    8644
Name: has_terrace, dtype: int64
True    3185
Name: has_balcony, dtype: int64
True    1556
Name: has_garden, dtype: int64
True    4772
Name: has_pool, dtype: int64
True    7155
Name: has_storage_room, dtype: int64
True    8644
Name: has_terrace, dtype: int64
True    3185
Name: has_balcony, dtype: int64
True    1556
Name: has_garden, dtype: int64
True    4772
Name: has_pool, dtype: int64


In [10]:
# Como podemos comprobar, los valores NaN en todos los casos anteriores pueden ser sustituidos por False.
# Vamos a proceder por tanto a cambiar esos datos por valores False

In [11]:
def fillNaN(columna):
    sales_houses[columna] = sales_houses[columna].fillna(value="False")
    return sales_houses[columna]

fillNaN ("has_storage_room")
fillNaN ("has_terrace")
fillNaN ("has_balcony")
fillNaN ("has_garden")
fillNaN ("has_pool")

0        False
1        False
2        False
3        False
4         True
         ...  
20705     True
20706     True
20707     True
20708    False
20709     True
Name: has_pool, Length: 20710, dtype: object

0        False
1        False
2        False
3        False
4         True
         ...  
20705     True
20706     True
20707     True
20708    False
20709     True
Name: has_pool, Length: 20710, dtype: object

In [12]:
# Volvemos a analizar los datos de nuestro DataFrame.

sales_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20710 entries, 0 to 20709
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               20710 non-null  int64  
 1   neighborhood_id     20710 non-null  object 
 2   house_type_id       20319 non-null  object 
 3   is_renewal_needed   20710 non-null  bool   
 4   is_new_development  19740 non-null  object 
 5   sq_mt_built         20585 non-null  float64
 6   n_rooms             20710 non-null  int64  
 7   n_bathrooms         20694 non-null  float64
 8   floor               18196 non-null  object 
 9   is_exterior         17687 non-null  object 
 10  has_lift            18330 non-null  object 
 11  has_parking         20710 non-null  bool   
 12  has_storage_room    20710 non-null  object 
 13  has_terrace         20710 non-null  object 
 14  has_balcony         20710 non-null  object 
 15  has_garden          20710 non-null  object 
 16  has_

In [13]:
null_columns=sales_houses.columns[sales_houses.isnull().any()]
sales_houses[null_columns].isnull().sum()
display(sales_houses[sales_houses.isnull().any(axis=1)][null_columns].head(10))

,house_type_id,is_new_development,sq_mt_built,n_bathrooms,floor,is_exterior,has_lift
8,HouseType 1: Pisos,False,84.0,2.0,NaN,True,True
11,HouseType 1: Pisos,False,122.0,2.0,6,NaN,True
15,HouseType 1: Pisos,False,115.0,1.0,3,True,NaN
23,HouseType 1: Pisos,False,51.0,1.0,Semi-sótano,NaN,NaN
24,HouseType 1: Pisos,False,134.0,2.0,2,NaN,True
25,HouseType 1: Pisos,NaN,62.0,1.0,2,True,NaN
30,HouseType 1: Pisos,NaN,75.0,1.0,1,True,True
32,HouseType 1: Pisos,NaN,61.0,1.0,2,True,NaN
37,HouseType 1: Pisos,False,60.0,1.0,NaN,True,False
38,HouseType 1: Pisos,NaN,78.0,1.0,2,True,True


,house_type_id,is_new_development,sq_mt_built,n_bathrooms,floor,is_exterior,has_lift
8,HouseType 1: Pisos,False,84.0,2.0,NaN,True,True
11,HouseType 1: Pisos,False,122.0,2.0,6,NaN,True
15,HouseType 1: Pisos,False,115.0,1.0,3,True,NaN
23,HouseType 1: Pisos,False,51.0,1.0,Semi-sótano,NaN,NaN
24,HouseType 1: Pisos,False,134.0,2.0,2,NaN,True
25,HouseType 1: Pisos,NaN,62.0,1.0,2,True,NaN
30,HouseType 1: Pisos,NaN,75.0,1.0,1,True,True
32,HouseType 1: Pisos,NaN,61.0,1.0,2,True,NaN
37,HouseType 1: Pisos,False,60.0,1.0,NaN,True,False
38,HouseType 1: Pisos,NaN,78.0,1.0,2,True,True


In [14]:
floor_isnull = sales_houses.floor.isnull()
sales_houses.loc[floor_isnull]

,index,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_garden,has_pool,buy_price,buy_price_by_area
8,8,Neighborhood 131: Butarque (2197.8 €/m2) - Dis...,HouseType 1: Pisos,False,False,84.0,3,2.0,NaN,True,True,True,True,False,False,False,True,212000,2524
37,37,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,60.0,2,1.0,NaN,True,False,False,False,True,False,False,False,154000,2567
50,52,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,HouseType 1: Pisos,False,False,65.0,2,1.0,NaN,True,False,False,False,False,False,False,False,115000,1769
82,85,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 2: Casa o chalet,True,False,150.0,2,1.0,NaN,NaN,NaN,True,True,True,False,False,False,159000,1060
125,130,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,HouseType 1: Pisos,False,NaN,54.0,2,1.0,NaN,NaN,False,False,False,True,False,False,False,136400,2526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20699,21730,Neighborhood 8: Alameda de Osuna (3113.42 €/m2...,HouseType 2: Casa o chalet,False,False,289.0,5,3.0,NaN,NaN,NaN,True,False,True,False,True,False,820000,2837
20700,21731,Neighborhood 11: Timón (2879.88 €/m2) - Distri...,HouseType 2: Casa o chalet,False,False,276.0,4,4.0,NaN,NaN,NaN,True,False,True,False,True,True,690000,2500
20702,21733,Neighborhood 9: Campo de las Naciones-Corralej...,HouseType 2: Casa o chalet,False,NaN,267.0,5,3.0,NaN,NaN,NaN,True,True,True,False,True,True,790000,2959
20708,21740,Neighborhood 9: Campo de las Naciones-Corralej...,HouseType 2: Casa o chalet,False,False,289.0,4,3.0,NaN,NaN,NaN,True,True,True,True,True,False,695000,2405


,index,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_garden,has_pool,buy_price,buy_price_by_area
8,8,Neighborhood 131: Butarque (2197.8 €/m2) - Dis...,HouseType 1: Pisos,False,False,84.0,3,2.0,NaN,True,True,True,True,False,False,False,True,212000,2524
37,37,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 1: Pisos,False,False,60.0,2,1.0,NaN,True,False,False,False,True,False,False,False,154000,2567
50,52,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,HouseType 1: Pisos,False,False,65.0,2,1.0,NaN,True,False,False,False,False,False,False,False,115000,1769
82,85,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,HouseType 2: Casa o chalet,True,False,150.0,2,1.0,NaN,NaN,NaN,True,True,True,False,False,False,159000,1060
125,130,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,HouseType 1: Pisos,False,NaN,54.0,2,1.0,NaN,NaN,False,False,False,True,False,False,False,136400,2526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20699,21730,Neighborhood 8: Alameda de Osuna (3113.42 €/m2...,HouseType 2: Casa o chalet,False,False,289.0,5,3.0,NaN,NaN,NaN,True,False,True,False,True,False,820000,2837
20700,21731,Neighborhood 11: Timón (2879.88 €/m2) - Distri...,HouseType 2: Casa o chalet,False,False,276.0,4,4.0,NaN,NaN,NaN,True,False,True,False,True,True,690000,2500
20702,21733,Neighborhood 9: Campo de las Naciones-Corralej...,HouseType 2: Casa o chalet,False,NaN,267.0,5,3.0,NaN,NaN,NaN,True,True,True,False,True,True,790000,2959
20708,21740,Neighborhood 9: Campo de las Naciones-Corralej...,HouseType 2: Casa o chalet,False,False,289.0,4,3.0,NaN,NaN,NaN,True,True,True,True,True,False,695000,2405


In [ ]:
sales_houses.head(10)

In [ ]:
sales_houses.info()

In [ ]:
for row in sales_houses.floor:
    print(row)
    if 

In [ ]:
sales_houses.floor = sales_houses.apply(lambda row: row["floor"].fillna(value="False") if )
sales_houses
#if [sales_houses["house_type_id"]=="HouseType 2: Casa o chalet"])

In [ ]:
sales_houses.loc[sales_houses["house_type_id"]=="HouseType 2: Casa o chalet"] = df.loc[df.a==1,'b'].fillna(df[df['a'] == 1]['b'].median())

In [ ]:
# Vamos a comprobar ahora aquellas columnas 

sales_houses["has_lift"].value_counts()

In [ ]:
sales_houses.isnull().any(axis=1).loc[:,"is_renewal_needed":"is_new_development"].head()

In [ ]:
sales_houses["is_new_development"].value_counts()

In [ ]:
sales_houses.info()

In [ ]:
sales_houses.house_type_id.value_counts()

In [16]:
sales_houses.neighborhood_id = sales_houses.neighborhood_id.str.findall(r"(\: \w.* \()(?:.+)(\: \w.*)").map(lambda x: x[-1])

In [17]:
#def cleaneigh():

sales_houses.neighborhood_id = sales_houses.neighborhood_id.apply(lambda neigh: neigh[0].lstrip(": ").rstrip(" (")+",", neigh[1].lstrip(": "))

#for neigh in sales_houses.neighborhood_id:
    #a = neigh[0].lstrip(": ").rstrip(" ("), neigh[1].lstrip(": ")
    #sales_houses.neighborhood_id.apply()
        #neigh = neigh[0].lstrip(": ").rstrip(" ("), neigh[1].lstrip(": ")
    #return sales_houses.neighborhood_id

NameError: name 'neigh' is not defined

NameError: name 'neigh' is not defined

In [ ]:
#sales_houses.neighborhood_id.replace(": ","")
sales_houses.neighborhood_id

In [ ]:
sales_houses.neighborhood_id[0][0].lstrip(": ").rstrip(" (")
sales_houses.neighborhood_id[0][1].lstrip(": ")

In [ ]:
# Saco los barrios.

sales_houses.neighborhood_id = sales_houses.neighborhood_id.str.extract("(\: \w.*\()")
sales_houses.neighborhood_id = sales_houses.neighborhood_id.str.lstrip(": ")
sales_houses.neighborhood_id = sales_houses.neighborhood_id.str.rstrip(" (")
sales_houses.neighborhood_id = sales_houses.neighborhood_id + ", Madrid"

In [ ]:
sales_houses.head()

In [ ]:
sales_houses.neighborhood_id.value_counts()

In [ ]:
saleproof = sales_houses.head(5)

In [ ]:
geolocator = Nominatim(user_agent="http")

location = [geolocator.geocode(neigh) for neigh in saleproof.neighborhood_id]
sales_houses["latitude"] = location.latitude
sales_houses["longitude"] = location.longitude

In [ ]:
sales_houses

In [ ]:
locs = [ geolocator.geocode(addr) for addr in saleproof.neighborhood_id ]
geo_info = pd.DataFrame(
    [ (addr.address, addr.latitude, addr.longitude) for addr in locs ],
    columns=['googleaddress', 'latitude', 'longitude'])

In [ ]:
geo_info